In [33]:
import pandas as pd

In [34]:
# Importacion de Plotly como libreria de DataViz
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [35]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


#### Leyendo datos:

In [42]:
# Generando el Dataframe apartir de los dataset RedTeam y BlueTeam
blueTeam = pd.read_csv( path_base + r"\\data\procesed\blueTeam.csv")
blueTeam["Team"] = "Blue"
redTeam = pd.read_csv( path_base + r"\\data\procesed\redTeam.csv")
redTeam["Team"] = "Red"
data = pd.concat([blueTeam,redTeam])

In [43]:
# Reemplazar los valores de la columna "Wins"
data['WinTags'] = data['Wins'].replace({0: 'Loss', 1: 'Win'})

# Crear un mapeo de colores
color_map = {'Win': 'yellow', 'Loss': 'gray'}

In [45]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [46]:
# Detalle de tabla
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19758 entries, 0 to 9878
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Wins                      19758 non-null  int64  
 1   WardsPlaced               19758 non-null  int64  
 2   WardsDestroyed            19758 non-null  int64  
 3   FirstBlood                19758 non-null  int64  
 4   Kills                     19758 non-null  int64  
 5   Deaths                    19758 non-null  int64  
 6   Assists                   19758 non-null  int64  
 7   EliteMonsters             19758 non-null  int64  
 8   Dragons                   19758 non-null  int64  
 9   Heralds                   19758 non-null  int64  
 10  TowersDestroyed           19758 non-null  int64  
 11  TotalGold                 19758 non-null  int64  
 12  AvgLevel                  19758 non-null  float64
 13  TotalExperience           19758 non-null  int64  
 14  TotalMi

### Hipotesis
Estas hipótesis proporcionan una base sólida para realizar pruebas y análisis más detallados sobre el dataset de League of Legends. Al evaluar cada una de estas hipótesis y validarlas con datos, el equipo podrá obtener una mejor comprensión de los factores que realmente influyen en el rendimiento y la probabilidad de victoria en el juego.

- **Hipótesis 1**: Control temprano del mapa y victoria Equipos que logran un mayor control del mapa mediante la colocación de más "Warding totems" en áreas estratégicas durante los primeros 10 minutos tendrán una mayor probabilidad de ganar la partida.

- **Hipótesis 1**: Relación entre asesinatos y tasa de victoria El número de asesinatos conseguidos por un equipo en los primeros 10 minutos se correlacionará positivamente con su tasa de victoria. Cuanto mayor sea la cantidad de asesinatos, más alta será la probabilidad de ganar.

In [51]:
# Crear un histograma combinado
fig = px.histogram(data, x='Kills',  color='WinTags', color_discrete_map=color_map, marginal="box", title="Histograma de Asesinatos en Partidas Ganadas y Perdidas")
fig.update_layout(xaxis_title="Número de Asesinatos", yaxis_title="Cantidad de Partidas")

# Mostrar el gráfico
fig.show()
fig.write_html(path_base + r"\\static\html\Hypothesis_001.html")
fig.write_image(path_base + r"\\static\img\Hypothesis_001.png")

- **Hipótesis 3**: Impacto de objetivos neutrales en la victoria Equipos que aseguran más dragones y Herald durante los primeros 10 minutos tendrán una mayor probabilidad de ganar la partida, ya que estos objetivos proporcionan beneficios significativos para todo el equipo.

In [73]:
pd.concat([data.loc[data["Dragons"]==1][["TotalExperience","TotalGold"]].describe(),
            data.loc[data["Dragons"]==0][["TotalExperience","TotalGold"]].describe()],axis=1)

,TotalExperience,TotalGold,TotalExperience,TotalGold
count,7657.000000,7657.000000,12101.000000,12101.000000
mean,18235.365940,16841.973488,17761.138584,16277.488307
std,1120.573841,1493.577057,1211.682355,1484.703329
min,13119.000000,12747.000000,10098.000000,10730.000000
25%,17497.000000,15788.000000,16999.000000,15227.000000
50%,18245.000000,16732.000000,17797.000000,16168.000000
75%,18992.000000,17788.000000,18575.000000,17194.000000
max,22269.000000,23701.000000,22224.000000,23424.000000


In [60]:
# Crear un Scatter combinado
fig = px.scatter(data, x="TotalExperience", y="TotalGold", 
                color='WinTags', color_discrete_map=color_map, 
                facet_row="Dragons", marginal_y="box",
                title="Correlación entre Dragon y Experiencia vs Oro")

# Mostrar el gráfico
fig.show()
fig.write_html(path_base + r"\\static\html\Hypothesis_002A.html")
fig.write_image(path_base + r"\\static\img\Hypothesis_002A.png")

In [61]:
# Crear un Scatter combinado
fig = px.scatter(data, x="TotalExperience", y="TotalGold", 
                color='WinTags', color_discrete_map=color_map, 
                facet_row="Heralds", marginal_y="box",
                title="Correlación entre Herald y Experiencia vs Oro")

# Mostrar el gráfico
fig.show()
fig.write_html(path_base + r"\\static\html\Hypothesis_002B.html")
fig.write_image(path_base + r"\\static\img\Hypothesis_002B.png")

In [69]:
# Crear un gráfico de heatmap para mostrar la relación
fig = px.density_heatmap(data.loc[data["WinTags"]=="Win"], x='Dragons', y='Heralds', text_auto=True,
                         title="Relación entre Captura de Dragones, Herald y Resultado")

# Personalizar el gráfico
fig.update_layout(
    xaxis_title="Cantidad de Dragones",
    yaxis_title="Cantidad de Heralds",
    coloraxis_colorbar_title="Resultado",
)

# Mostrar el gráfico
fig.show()
fig.write_html(path_base + r"\\static\html\Hypothesis_002.html")
fig.write_image(path_base + r"\\static\img\Hypothesis_002.png")

In [68]:
h1d1 = 1043
h1d0 = 1030
h0d1 = 3803
h0d0 = 4003

suma = h1d1+h1d0+h0d1+h0d0
print("evaluacion:")
print("no dragon y no herald:", round(h0d0/suma,4))
print("si dragon y no herald:", round(h0d1/suma,4))
print("no dragon y si herald:", round(h1d0/suma,4))
print("si dragon y si herald:", round(h1d1/suma,4))


evaluacion:
no dragon y no herald: 0.4052
si dragon y no herald: 0.385
no dragon y si herald: 0.1043
si dragon y si herald: 0.1056


- **Hipótesis 4**: Importancia de la ventaja económica y de experiencia Equipos que logran una ventaja significativa en oro y experiencia sobre el equipo contrario en los primeros 10 minutos tendrán mayores posibilidades de obtener la victoria.


- **Hipótesis 5**: Contribución individual y tasa de victoria Jugadores que contribuyen con un mayor número de asistencias tendrán un impacto positivo en la tasa de victoria de su equipo en las partidas clasificatorias Diamante.

- **Hipótesis 6**: Roles clave para el éxito Identificar roles específicos, como un jungla efectivo o un carry con muchas asistencias, será crucial para el éxito del equipo y aumentará sus posibilidades de ganar.

- **Hipótesis 7**: Nivel promedio y probabilidad de victoria Un equipo con un mayor nivel promedio de sus jugadores durante los primeros 10 minutos tendrá una mayor probabilidad de ganar la partida.

- **Hipótesis 8**: Adaptabilidad y éxito Equipos que pueden ajustar sus tácticas y estrategias según la situación en los primeros 10 minutos tendrán una mayor probabilidad de lograr la victoria en partidas clasificatorias Diamante.

In [ ]:
# Crear el gráfico de dispersión
fig = px.scatter(data, x='GoldDiff', y='ExperienceDiff', color='WinTags', color_discrete_map=color_map,
                 labels={'GoldDiff': 'Diferencia de Oro', 'ExperienceDiff': 'Diferencia de Experiencia', 'WinTags': 'Resultado de la Partida'},
                 title="Relación entre Diferencia de Oro, Diferencia de Experiencia y Resultado de la Partida")

# Personalizar el gráfico
fig.update_layout(
    xaxis_title="Diferencia de Oro",
    yaxis_title="Diferencia de Experiencia"
)

# Cambiar las etiquetas de la leyenda
fig.update_traces(marker=dict(size=6), selector=dict(name=0), legendgroup="Loss", showlegend=True)
fig.update_traces(marker=dict(size=6), selector=dict(name=1), legendgroup="Win", showlegend=True)

# Mostrar el gráfico
fig.show()